<h2>SMS Spam Detection</h2>
See <a href="https://www.kaggle.com/uciml/sms-spam-collection-dataset">Kaggle.com</a>

In [4]:
import numpy as np
import numpy.core.defchararray as npf
from sklearn.preprocessing import LabelEncoder

# load data
import pandas as pd
#sms = pd.read_csv("../data/spam.csv", encoding="latin-1")
sms = pd.read_csv("../data/spam_utf8.csv")
sms = sms.drop(['Unnamed: 2','Unnamed: 3','Unnamed: 4'],axis=1)
sms = sms.rename(columns = {'v1':'label','v2':'message'})

X_raw = sms["message"]
y_raw = sms["label"]

print(X_raw[5])
print(y_raw[5])

for i in range(0,X_raw.shape[0]):
    X_raw[i] = str(npf.replace(X_raw[i], "'", ""))
    X_raw[i] = str(npf.replace(X_raw[i], "å", ""))
    X_raw[i] = str(npf.replace(X_raw[i], "!", ""))
    X_raw[i] = str(npf.replace(X_raw[i], "?", ""))
    X_raw[i] = str(npf.replace(X_raw[i], ".", " "))
    X_raw[i] = str(npf.replace(X_raw[i], "\"", ""))

print(X_raw[5])
    
# Convert class label strings to integers
encoder = LabelEncoder()
encoder.fit(y_raw)
y = encoder.transform(y_raw)

#np_data = df.values

# split data into X and y
#X_raw = np_data[:,0:-1]
#y_raw = pd.factorize(np_data[:,-1])[0]

#print(X_raw[4])
#print(y_raw[2])

# set seed to randomizer
#seed = 7

# flatten input matrix to vector
#X_raw = X_raw.ravel()
print("Instances: {}".format(X_raw.shape[0]))

FreeMsg Hey there darling it's been 3 week's now and no word back! I'd like some fun you up for it still? Tb ok! XxX std chgs to send, å£1.50 to rcv
spam
FreeMsg Hey there darling its been 3 weeks now and no word back Id like some fun you up for it still Tb ok XxX std chgs to send, £1 50 to rcv
Instances: 5572


<h2>Convert to bag of words</h2>

In [5]:
from sklearn.feature_extraction.text import CountVectorizer

def gen_bagofwords(X_raw, ng_min=1, ng_max=1, df_min=1):
    count_vect = CountVectorizer(ngram_range=(ng_min,ng_max), min_df=df_min)
    X = count_vect.fit_transform(X_raw)
    print("Bag-of-words size: {}".format(X.shape[1]))
    return X

<h2>Convert from occurences to frequencies</h2>

In [6]:
from sklearn.feature_extraction.text import TfidfTransformer

def conv_frequencies(X):
    tf_transformer = TfidfTransformer(sublinear_tf=True).fit(X)
    X = tf_transformer.transform(X)
    return X

<h2>Function for evaluating model accuracy</h2>

In [9]:
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split

def evaluate_test(model):
    print("\n-- Test set --")
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=111, stratify=y)
    # train model on training dataset
    model.fit(X_train, y_train)
    # evaluate dataset
    y_pred = model.predict(X_test)
    # calculate accuracy
    accuracy = accuracy_score(y_test, y_pred)
    print("Accuracy: %.2f%%" % (accuracy * 100.0))
    # confusion matrix
    print("Confusion Matrix:")
    conf_mx = confusion_matrix(y_test, y_pred)
    print(conf_mx)
       
def evaluate_cv(model):
    print("\n-- 5-fold CV --")
    # 10-fold CV
    y_pred = cross_val_predict(model, X, y, cv=5)
    # calculate accuracy
    accuracy = accuracy_score(y, y_pred)
    print("Average accuracy: %.2f%%" % (accuracy * 100.0))
    # confusion matrix
    print("Confusion Matrix:")
    conf_mx = confusion_matrix(y, y_pred)
    print(conf_mx)

<h2>Naive Bayes</h2>

In [10]:
from sklearn.naive_bayes import BernoulliNB, MultinomialNB

#X = gen_bagofwords(X_raw, df_min=0.001)
X = gen_bagofwords(X_raw)
X = conv_frequencies(X)

model = MultinomialNB(alpha=0.01)
evaluate_test(model)
evaluate_cv(model)

Bag-of-words size: 8813

-- Test set --
Accuracy: 98.83%
Confusion Matrix:
[[959   7]
 [  6 143]]

-- 5-fold CV --
Average accuracy: 98.47%
Confusion Matrix:
[[4787   38]
 [  47  700]]


In [177]:
X = gen_bagofwords(X_raw, 1, 2, df_min=0.001)
#X = gen_bagofwords(X_raw)
X = conv_frequencies(X)

model = MultinomialNB(alpha=0.01)
evaluate_test(model)
evaluate_cv(model)

Bag-of-words size: 3064

-- Test set --
Accuracy: 98.65%
Confusion Matrix:
[[963   3]
 [ 12 137]]

-- 5-fold CV --
Average accuracy: 98.46%
Confusion Matrix:
[[4812   13]
 [  73  674]]


<h2>SVM</h2>

In [178]:
from sklearn import svm

X = gen_bagofwords(X_raw, df_min=0.001)
#X = gen_bagofwords(X_raw)
X = conv_frequencies(X)

model = svm.LinearSVC(random_state=42)
evaluate_test(model)
evaluate_cv(model)

Bag-of-words size: 1572

-- Test set --
Accuracy: 98.92%
Confusion Matrix:
[[963   3]
 [  9 140]]

-- 5-fold CV --
Average accuracy: 98.60%
Confusion Matrix:
[[4814   11]
 [  67  680]]


In [179]:
X = gen_bagofwords(X_raw, 1, 2, df_min=0.001)
#X = gen_bagofwords(X_raw)
X = conv_frequencies(X)

model = svm.LinearSVC(random_state=42)
evaluate_test(model)
evaluate_cv(model)

Bag-of-words size: 3064

-- Test set --
Accuracy: 99.10%
Confusion Matrix:
[[964   2]
 [  8 141]]

-- 5-fold CV --
Average accuracy: 98.76%
Confusion Matrix:
[[4817    8]
 [  61  686]]


<h2>Pipeline example</h2>

In [7]:
from sklearn.pipeline import Pipeline
X = X_raw.ravel()
model = Pipeline([('vect', CountVectorizer(stop_words='english')),
                  ('tfidf', TfidfTransformer()),
                  ('clf', MultinomialNB(alpha=.01)),])
evaluate(model)

-- Training data --
Accuracy: 99.67%
Confusion Matrix:
[[150   0]
 [  1 149]]

-- 10-fold CV --
Average accuracy: 95.33%
Confusion Matrix:
[[144   6]
 [  8 142]]
